# Подсчет процента схожести описаний товаров (сравнение нечетких строк)

## 1. Описание
Задача скрипта - подсчитать процент схожести описаний товаров внутри одной товарной группы.

На вход передается таблица со следующими столбцами:
* `code` - код товарной группы;
* `description` - описание товара;
* `sim_perc` - процент схожести (пустой столбец).

In [1]:
import pandas as pd
import difflib as df
import numpy as np

In [2]:
# Импорт тестовых данных
data = pd.read_excel('dataset.xlsx')
data

,code,description,sim_perc
0,1,Чай 100гр,NaN
1,1,Кофе 100гр,NaN
2,2,Кофе 200гр,NaN
3,2,Кофе 900гр,NaN
4,2,Кофе 300гр,NaN
5,3,Сосиски говяжьи вареные,NaN
6,3,Сосиски говяжьи вареные,NaN
7,3,Сосиски говяжьи вареные,NaN
8,4,Чай,NaN
9,4,rerr rrrr rr,NaN


## 2. Функция расчета % схожести

Снача описания товаров группы выделяются в список, затем его элементы поочередно сравниваются между собой. В итоге работы функция возвращает среднее значение процента схожести в группе.<br>
В функции предусмотрено удаление лишних пробелов, поэтому они не будут вносить искажение в рассчитанный процент.

In [6]:
def group_comp(data, code):
    
    # Создаем список из описаний товаров, входящих в группу
    cat_group = data.query('code == @code')['description'].tolist()
    
    # Удаляем лишние пробелы в элементах списка
    def delete_spaces(cat_group):
        
        clean_cat_group = []
        for item in cat_group:
            clean_cat_group.append(' '.join(item.split()))
        return clean_cat_group
    
    cat_group = delete_spaces(cat_group)
    
    # Сравниваем элементы списка по порядку, начиная с первого, затем считаем средний процент схожести для всей
    # группы как среднее результатов сравнения каждой пары
    total_group_perc = 0
    
    for i in range(len(cat_group) - 1):
        total_group_perc += df.SequenceMatcher(lambda x: x == ' ',
                                               cat_group[i].lower(), cat_group[i + 1].lower()).ratio()
        # Возможно использование метода .quick_ratio(), который возвращает верхнюю границу сравнения - 
        # фактически, сравнивает наличие одинаковых букв в строках, все зависимости от их расположения
    
    group_sim_perc = total_group_perc / (len(cat_group) - 1) * 100 # Находим средний процент для группы
    
    return str(int(group_sim_perc)) + '%'

## 3. Вывод

In [7]:
# Применяем функцию group_comp() к каждой группе
data['sim_perc'] = np.NaN
for code in data['code'].unique():
    data[data['code'] == code] = data.fillna(group_comp(data, code))
data

,code,description,sim_perc
0,1,Чай 100гр,63%
1,1,Кофе 100гр,63%
2,2,Кофе 200гр,90%
3,2,Кофе 900гр,90%
4,2,Кофе 300гр,90%
5,3,Сосиски говяжьи вареные,100%
6,3,Сосиски говяжьи вареные,100%
7,3,Сосиски говяжьи вареные,100%
8,4,Чай,0%
9,4,rerr rrrr rr,0%


При необходимости можно убрать комментирование и вывести полученную таблицу в папку блокнота в формате `.xlsx`.

In [5]:
# data.to_excel(r'dataset_filled.xlsx', index = False)